# Demonstrate allocated score voting

See https://github.com/nealmcb/allocated_score_voting

In [1]:
import pandas as pd
import allocated_score

In [2]:
max_score = 5
num_winners = 3

Example from star-core example at https://github.com/Equal-Vote/star-core

In [3]:
candidates = 'Adam,Becky,Cindy,Dylan,Eliza'.split(',')

In [4]:
scores = [
    [0,0,5,3,2],
    [4,0,3,3,2],
    [0,0,0,3,1],
    [2,0,0,3,4],
    [0,0,0,0,0],
    [1,5,0,3,5],
    [0,0,0,0,0],
]

In [5]:
S = pd.DataFrame.from_records(scores, columns=candidates)

In [6]:
S

,Adam,Becky,Cindy,Dylan,Eliza
0,0,0,5,3,2
1,4,0,3,3,2
2,0,0,0,3,1
3,2,0,0,3,4
4,0,0,0,0,0
5,1,5,0,3,5
6,0,0,0,0,0


In [7]:
allocated_score.Allocated_Score(max_score, num_winners, S)

['Dylan', 'Eliza', 'Adam']

## Example of non-monotonicity
This also shows how to set up an election with factions, each with as many candidates as there are seats, where voters are very simplistically modeled into `profiles` showing how they score the factions, and in which everyone ranks every candidate in the faction the same way.

In [8]:
num_winners = 5

In [9]:
factions = ['A', 'B', 'C']
candidates = allocated_score.dup_factions(factions, num_winners)

In [10]:
red   = [5, 0, 0]
green = [0, 4, 5]
blue  = [0, 3, 0]

In [11]:
profiles = [(red, 21), (green, 41), (blue, 38)]

With the blue profile of voters giving a score of 3 to the `B` faction, they get 4 winners:

In [12]:
allocated_score.tabulate_factions(max_score, num_winners, factions, candidates, profiles)

Example election: num_winners=5, max_score=5, factions=['A', 'B', 'C']
Profiles:
  21: [5, 0, 0]
  41: [0, 4, 5]
  38: [0, 3, 0]
Winners: ['B1', 'B2', 'B3', 'A1', 'B4']


But if blue gives a higher score of 5 to `B`, they only get 3 winners:

In [13]:
blue5  = [0, 5, 0]
profiles5 = [(red, 21), (green, 41), (blue5, 38)]
allocated_score.tabulate_factions(max_score, num_winners, factions, candidates, profiles5)

Example election: num_winners=5, max_score=5, factions=['A', 'B', 'C']
Profiles:
  21: [5, 0, 0]
  41: [0, 4, 5]
  38: [0, 5, 0]
Winners: ['B1', 'B2', 'B3', 'A1', 'C1']
